In [2]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken chromadb lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.9 MB/s eta 0:0

In [3]:
!pip -q install sentence_transformers
!pip -q install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 359.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [6]:
!mkdir -p Data
!unzip -q /content/langchain_blog_posts.zip -d Data

In [36]:
import os

os.environ["OPENAI_API_KEY"] = ""

# **Hypothetical Document Embeddings (HyDE)**

HyDE is a technique that uses a large language model to generate hypothetical answers, which can then be used for better retrieval in a RAG system

In [37]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

from langchain.document_loaders import TextLoader
import langchain

In [38]:
llm = OpenAI()

embeddings = HypotheticalDocumentEmbedder.from_llm(llm,
                                                   OpenAIEmbeddings(),
                                                   prompt_key="web_search"
                                                   )

In [39]:
embeddings.llm_chain.prompt

PromptTemplate(input_variables=['QUESTION'], template='Please write a passage to answer the question \nQuestion: {QUESTION}\nPassage:')

In [40]:
langchain.debug = True

In [41]:
result = embeddings.embed_query("What items does McDonalds make?")

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: What items does McDonalds make?\nPassage:"
  ]
}
[llm/end] [1:llm:OpenAI] [2.16s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " McDonalds is a popular fast food chain that offers a wide range of food items on their menu. Some of the most popular items that McDonalds makes include their iconic Big Mac burger, crispy and golden French fries, chicken McNuggets, and Filet-O-Fish sandwich. These items have become staples in the diet of many people around the world and are synonymous with the McDonalds brand. In addition to these classic items, McDonalds also offers a variety of other burgers, sandwiches, salads, and breakfast items. They also have a selection of desserts, such as their famous apple pie and McFlurry. McDonalds is constantly innovating and adding new items to their menu, such as their plant-based burger, th

# **Multiple Documents / Generation**

In [44]:
llms = OpenAI(n=4, best_of=4)

embeddings = HypotheticalDocumentEmbedder.from_llm(
    llms,
    embeddings,
    "web_search"
)

In [45]:
result = embeddings.embed_query("What is McDonalds best selling item?")

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: What is McDonalds best selling item?\nPassage:"
  ]
}
[llm/end] [1:llm:OpenAI] [2.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " McDonald's is a global fast food chain that is known for its wide range of menu offerings. From burgers and fries to salads and breakfast items, there is something for everyone at McDonald's. However, when it comes to their best-selling item, one item stands out among the rest - the Big Mac. This iconic burger, with its two all-beef patties, special sauce, lettuce, cheese, pickles, and onions on a sesame seed bun, has been a fan favorite since its introduction in 1967. It has become a symbol of McDonald's and is recognized worldwide. The Big Mac has a perfect combination of flavors and textures that keep customers coming back for more. It has even inspired spin-off items such as the Mac

# **Own Prompt**

In [46]:
prompt_template = """Please answer the user's question as a single food item
Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["question"], template=prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [47]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=embeddings
)

In [48]:
result = embeddings.embed_query(
    "What is is McDonalds best selling item?"
)

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as a single food item\nQuestion: What is is McDonalds best selling item?\nAnswer:"
  ]
}
[llm/end] [1:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " McDonald's best selling item is the Big Mac.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 25,
      "total_tokens": 35,
      "completion_tokens": 10
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}


In [50]:
print(result)

[-0.022104659721515995, -0.002728893381151363, -0.01163043250789999, -0.011394218372832957, 0.0012377925266556105, -0.012276912857456976, -0.033194284394221805, -0.015577692307656897, -0.0025051117037792805, -0.010200715472633475, 0.007969114811344801, 0.042717441283443916, 0.022850596938664907, -0.017231190321803577, -0.01628633378153544, 0.025175440683419755, 0.027798659687563194, -0.013874465394052975, 0.018399828772274435, -0.003087876551660711, -0.0010404294435355092, 0.00516873543873173, -0.017877671602683123, -0.015739312162215503, 0.025585706897909708, 0.015266884823403996, 0.004152393838324444, -0.01251312699252401, 0.017803076390852133, -0.006962097146755109, -0.006688586337095142, 0.0005757716631875955, -0.023770589029204422, 0.008534785233622486, -0.023173838510427244, 0.010399632933107575, -0.0088269448462402, -0.012904744404056215, 0.03563101909407801, 0.005731298037623974, 0.016584710903569153, 0.003850910057058495, 0.013712845539494369, 0.0006674600200538112, -0.0266548

# **Using HYDE**

In [51]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loaders = [
    TextLoader('/content/Data/blog.langchain.dev_announcing-langsmith_.txt'),
    TextLoader('/content/Data/blog.langchain.dev_benchmarking-question-answering-over-csv-data_.txt'),
    TextLoader('/content/Data/blog.langchain.dev_chat-loaders-finetune-a-chatmodel-in-your-voice_.txt'),
]

In [52]:
docs = []
for l in loaders:
    docs.extend(l.load())

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

texts = text_splitter.split_documents(docs) #split_text

In [53]:
texts

[Document(page_content='URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.', metadata={'source': '/content/Data/blog.langchain.dev_announcing-langsmith_.txt'}),
 Document(page_content='The blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of 

In [54]:
prompt_template = """Please answer the user's question as related to Large Language Models
Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["question"], template=prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [55]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=embeddings
)

In [56]:
docsearch = Chroma.from_documents(texts, embeddings)

query = "What are chat loaders?"
docs = docsearch.similarity_search(query)

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as related to Large Language Models\nQuestion: What are chat loaders?\nAnswer:"
  ]
}
[llm/end] [1:llm:OpenAI] [1.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Chat loaders are a type of Large Language Model (LLM) designed to assist with generating human-like responses in conversational applications, such as chatbots. These models are trained on vast amounts of text data and use Artificial Intelligence (AI) to understand and respond to user input in a natural and coherent manner. Chat loaders are often used in customer service, virtual assistants, and other chat-based applications to enhance the user experience and provide more personalized interactions.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usa

In [57]:
print(docs[0].page_content)

Why is this better than direct instructions? Style and tone can be hard to describe! Most of us don't write like ChatGPT, and it can sometimes be frustratingly difficult to get the LLM to consistently respond in a particular voice (especially over longer conversations).

Why is this better than few-shot examples? It can be challenging to capture your voice in only a few concise snippets! Fine-tuning lets you provide a larger number of examples the model can learn from without having to see them every time you want to query the model.

ChatLoaders

At LangChain, we want to make it as easy as possible for you to take advantage of this improved fine-tuning support. To make it simple to adapt a model to your voice, we're adding a new integration type: ChatLoaders .
